In [16]:
#imports
import pandas as pd
from google.cloud import bigquery
from time import time
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "/Users/reneboygarcia/Library/CloudStorage/GoogleDrive-reneboygarcia@gmail.com/My Drive/Personal/Data Science Notebook/Data Engineering/week_3_data_warehouse/GOOGLE_APPLICATION_CREDENTIALS.json"
print("Setup Complete")

Setup Complete


In [4]:
#create a client
client = bigquery.Client()
# Construct a reference to the "prefect_de_2023_datasetid" dataset
dataset_ref = client.dataset("prefect_de_2023_datasetid", project="dtc-de-2023")
# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)
#construct a table reference to "yellow_ny_taxi_trips_table"
table_ref = dataset_ref.table('yellow_ny_taxi_trips_table')
#API request
table_yellow_taxi_trips = client.get_table(table_ref)
#to dataframe
df_yellow_taxi_trips = client.list_rows(table=table_yellow_taxi_trips, max_results=5).to_dataframe()
df_yellow_taxi_trips.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,pickup_date,dropoff_date
0,1,2021-01-01 00:43:30+00:00,2021-01-01 01:11:06+00:00,1.0,14.70,1.0,N,132,165,1,...,0.5,0.5,8.65,0.0,0.3,51.95,0.0,NaN,2021-01-01,2021-01-01
1,1,2021-01-01 00:15:48+00:00,2021-01-01 00:31:01+00:00,0.0,10.60,1.0,N,138,132,1,...,0.5,0.5,6.05,0.0,0.3,36.35,0.0,NaN,2021-01-01,2021-01-01
2,2,2021-01-01 00:19:57+00:00,2021-01-01 00:43:03+00:00,3.0,10.74,1.0,N,264,231,1,...,0.5,0.5,4.00,0.0,0.3,40.30,2.5,NaN,2021-01-01,2021-01-01
3,2,2021-01-01 00:44:58+00:00,2021-01-01 01:07:41+00:00,2.0,5.85,1.0,N,249,238,1,...,0.5,0.5,5.06,0.0,0.3,30.36,2.5,NaN,2021-01-01,2021-01-01
4,2,2021-01-01 00:06:11+00:00,2021-01-01 00:23:40+00:00,1.0,8.06,1.0,N,75,264,1,...,0.5,0.5,5.66,0.0,0.3,33.96,2.5,NaN,2021-01-01,2021-01-01


In [59]:
# query
query = """
        SELECT COUNT(1) as num_count
        FROM `dtc-de-2023.prefect_de_2023_datasetid.yellow_ny_taxi_trips_table`
        WHERE DATE(tpep_pickup_datetime) BETWEEN "2021-01-01" AND "2021-01-01"
        LIMIT 100
        """
        
#limit results to 10GB=10**10
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed = 10**10)
#query
query_results = client.query(query, job_config=safe_config)
#convert to dataframe
df_query_results = query_results.to_dataframe()
df_query_results.head()

,num_count
0,24827


(1, 1)

In [22]:
# amount of data scanned in GB
def data_scanned(query):
    dry_run_config = bigquery.QueryJobConfig(dry_run=True)
    query_job_bytes = client.query(query, job_config=dry_run_config).total_bytes_processed
    print(f'Data processed: {round(query_job_bytes / 10**9, 3)} GB')

def runtime(query):
    time_config = bigquery.QueryJobConfig(use_query_cache=False)
    start = time()
    query_result = client.query(query, job_config=time_config).result()
    end = time()
    print(f'Runtime is: {round(end-start, 3)} seconds')

In [23]:
data_scanned(query=query)
runtime(query=query)

Data processed: 0.011 GB
Runtime is: 1.568 seconds


In [52]:
# Construct a reference to the "prefect_de_2023" dataset
dataset_ref_gcs = bigquery.DatasetReference(project="dtc-de-2023", dataset_id="prefect-de-2023")
# construct a table reference to "green_tripdata_2019-04"
table_ref_gcs = dataset_ref_gcs.table("green_tripdata_2020-01")
# API request
green_ny_taxi_trips_table = bigquery.Table(table_ref_gcs)
external_config = bigquery.ExternalConfig("PARQUET")
external_config.source_uris = ["gs://prefect-de-2023/data/green/green_tripdata_2020-01.parquet"]
# green_ny_taxi_trips_table.external_data_configuration = external_config

# Create a permanent table linked to GCS file
table = client.create_table(table=green_ny_taxi_trips_table, )
# to DataFrame
df_green_ny_taxi_trips = client.list_rows(table=table, max_results=5).to_dataframe()
df_green_ny_taxi_trips.head()

# 404 POST https://bigquery.googleapis.com/bigquery/v2/projects/dtc-de-2023/datasets/prefect-de-2023/tables?prettyPrint=false: Not found: Dataset dtc-de-2023:prefect-de-2023

NotFound: 404 POST https://bigquery.googleapis.com/bigquery/v2/projects/dtc-de-2023/datasets/prefect-de-2023/tables?prettyPrint=false: Not found: Dataset dtc-de-2023:prefect-de-2023

In [ ]:
#create a client
client = bigquery.Client()
# Construct a reference to the "prefect_de_2023_datasetid" dataset
dataset_ref = client.dataset("prefect_de_2023_datasetid", project="dtc-de-2023")
# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)
# construct a table reference to "yellow_ny_taxi_trips_table"
table_ref = dataset_ref.table('yellow_ny_taxi_trips_table')
#API request
table_yellow_taxi_trips = client.get_table(table_ref)
#to dataframe
df_yellow_taxi_trips = client.list_rows(table=table_yellow_taxi_trips, max_results=5).to_dataframe()
df_yellow_taxi_trips.head()